In [1]:
import pandas as pd
from tqdm import tqdm
import imodels
from imodels import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../data/ICPSR_03986/DS0001/data_clean.csv')
X, y = df.drop(['id', 'any_deviance'], axis=1), df['any_deviance']

# different models select different rules

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
models = [
    BoostedRulesClassifier(n_estimators=5),
    RuleFitClassifier(n_estimators=10, tree_size=3, max_rules=5),
    DecisionTreeClassifier(max_depth=3)
]
for m in tqdm(models):
    try:
        m.fit(X_train, y_train, feature_names=X_train.columns)
    except:
        m.fit(X_train, y_train)
    print(m, 'acc', accuracy_score(y_test, m.predict(X_test)))
    try:
        print('\t', m.rules_)
    except:
        pass

  0%|          | 0/3 [00:00<?, ?it/s]

Mined rules:
	you_gang_fight <= 1.5
	you_broken_+_entering <= 1.5
	any_victimization <= 0.5
	you_attack_intent_kill/maim <= 1.5
	fr_suggest_agnts_law <= 1.5
 acc 0.9428571428571428
	 [you_gang_fight <= 1.5, you_gang_fight > 1.5, you_broken_+_entering <= 1.5, you_broken_+_entering > 1.5, any_victimization <= 0.5, any_victimization > 0.5, you_attack_intent_kill/maim <= 1.5, you_attack_intent_kill/maim > 1.5, fr_suggest_agnts_law <= 1.5, fr_suggest_agnts_law > 1.5]


100%|██████████| 3/3 [08:16<00:00, 165.34s/it]

RuleFitClassifier(max_rules=5, n_estimators=10, tree_size=3) acc 0.978021978021978
	 [you_attack_intent_kill/maim > 1.5 and you_broken_+_entering > 1.5, you_attack_intent_kill/maim > 1.5 and you_gang_fight > 1.5, you_broken_+_entering > 1.5 and you_gang_fight > 1.5, you_broken_+_entering > 1.5 and you_gang_fight > 1.5, you_attack_intent_kill/maim > 1.5 and you_broken_+_entering > 1.5]
DecisionTreeClassifier(max_depth=3) acc 0.9747252747252747


# the same model with slight bootstrap sampling selects different rules

In [4]:
np.random.seed(13)
for i in tqdm(range(3)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43 + i)
    m = imodels.BayesianRuleListClassifier(max_iter=10, minsupport=0.12, random_state=43 + i)
    m.fit(X_train, y_train, feature_names=X_train.columns)
    print('acc', accuracy_score(y_test, m.predict(X_test)))

 33%|███▎      | 1/3 [06:50<13:40, 410.38s/it]

acc 0.5769230769230769
Trained RuleListClassifier 
IF other_race_w/o_refused_0.0 AND fr_drunk_once_in_while_-inf_to_1.5 THEN probability of class 1: 23.3% (21.0%-25.7%)
ELSE IF beaten_with_fist/hurt_badly_1.5_to_inf AND sedatives_1.5_to_inf THEN probability of class 1: 4.2% (3.2%-5.3%)
ELSE IF experimental_marijuana_0.0 THEN probability of class 1: 31.9% (21.8%-43.1%)
ELSE IF anyone_childhood_count_on_All THEN probability of class 1: 75.0% (42.1%-96.3%)
ELSE probability of class 1: 50.0% (2.5%-97.5%)

rules None


 67%|██████▋   | 2/3 [13:22<06:39, 399.68s/it]

acc 0.6725274725274726
Trained RuleListClassifier 
IF friends_use_mar/hash_1.5_to_inf AND school_shooting_0.0 THEN probability of class 1: 4.7% (3.8%-5.8%)
ELSE IF fam_income_All AND you_stolen_motor_veh_1.5_to_inf THEN probability of class 1: 21.9% (19.5%-24.5%)
ELSE IF school_shooting_0.0 THEN probability of class 1: 98.1% (93.2%-100.0%)
ELSE IF total_school_sexual_assaults_0.0 THEN probability of class 1: 50.0% (2.5%-97.5%)
ELSE probability of class 1: 50.0% (2.5%-97.5%)

rules None


100%|██████████| 3/3 [19:47<00:00, 395.81s/it]

acc 0.6791208791208792
Trained RuleListClassifier 
IF seen_thre_w/gun_knife_1.5_to_inf AND without_weapon_1.5_to_inf THEN probability of class 1: 4.4% (3.5%-5.5%)
ELSE IF age_14.5_to_inf THEN probability of class 1: 33.4% (29.8%-37.1%)
ELSE IF non-exper_past_year_illicit_0.0 AND you_stolen_more_$100_1.5_to_inf THEN probability of class 1: 17.8% (14.2%-21.8%)
ELSE IF friends_stolen_less_$5_-inf_to_1.5 THEN probability of class 1: 88.9% (71.3%-98.5%)
ELSE probability of class 1: 66.7% (15.8%-98.7%)

rules None


In [7]:
np.random.seed(13)
for i in tqdm(range(3)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43 + i)
    m = imodels.BayesianRuleListClassifier(max_iter=100, minsupport=0.12, random_state=43 + i)
    m.fit(X_train, y_train, feature_names=X_train.columns)
    print('acc', accuracy_score(y_test, m.predict(X_test)))
    print('rules', print(m))

 33%|███▎      | 1/3 [07:35<15:10, 455.22s/it]

acc 0.5978021978021978
Trained RuleListClassifier 
IF my_check_of_wv_0.0 AND hispanic_0.0 THEN probability of class 1: 2.3% (1.3%-3.5%)
ELSE IF any_victimization_0.0 AND current_ptsd_0.0 THEN probability of class 1: 3.5% (2.3%-5.0%)
ELSE IF you_gang_fight_1.5_to_inf THEN probability of class 1: 15.0% (13.0%-17.1%)
ELSE probability of class 1: 99.4% (97.7%-100.0%)

rules None


 67%|██████▋   | 2/3 [14:42<07:18, 438.50s/it]

acc 0.8219780219780219
Trained RuleListClassifier 
IF age_-inf_to_14.5 AND friends_destroyed_property_1.5_to_inf THEN probability of class 1: 3.0% (2.0%-4.2%)
ELSE IF any_victimization_0.0 AND friends_destroyed_property_1.5_to_inf THEN probability of class 1: 2.9% (1.5%-4.6%)
ELSE IF you_attack_intent_kill/maim_1.5_to_inf AND fr_suggest_agnts_law_1.5_to_inf THEN probability of class 1: 9.4% (7.4%-11.5%)
ELSE IF you_attack_intent_kill/maim_1.5_to_inf AND beaten_with_fist/hurt_badly_1.5_to_inf THEN probability of class 1: 23.6% (19.5%-27.9%)
ELSE probability of class 1: 86.7% (81.5%-91.2%)

rules None


100%|██████████| 3/3 [21:42<00:00, 434.05s/it]

acc 0.8791208791208791
Trained RuleListClassifier 
IF seen_stab_1.5_to_inf AND you_broken_+_entering_1.5_to_inf THEN probability of class 1: 6.1% (5.2%-7.1%)
ELSE IF you_attack_intent_kill/maim_1.5_to_inf AND spanked_marks_1.5_to_inf THEN probability of class 1: 45.6% (39.5%-51.8%)
ELSE IF 1_failing_grade_1.5_to_inf THEN probability of class 1: 71.0% (54.1%-85.3%)
ELSE IF 1_failing_grade_-inf_to_1.5 AND friends_alcohol_1.5_to_inf THEN probability of class 1: 38.5% (15.2%-65.1%)
ELSE IF hard_drug_ab_dep_0.0 THEN probability of class 1: 90.1% (83.2%-95.3%)
ELSE probability of class 1: 90.0% (66.4%-99.7%)

rules None


In [8]:
np.random.seed(13)
for i in tqdm(range(3)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43 + i)
    m = imodels.BayesianRuleListClassifier(max_iter=500, minsupport=0.12, random_state=43 + i)
    m.fit(X_train, y_train, feature_names=X_train.columns)
    print('acc', accuracy_score(y_test, m.predict(X_test)))
    print('rules', print(m))

 33%|███▎      | 1/3 [07:38<15:17, 458.55s/it]

acc 0.589010989010989
Trained RuleListClassifier 
IF any_victimization_0.0 AND sedatives_1.5_to_inf THEN probability of class 1: 2.7% (1.9%-3.6%)
ELSE IF #_in_household_All AND you_broken_+_entering_1.5_to_inf THEN probability of class 1: 16.9% (14.9%-19.0%)
ELSE IF city/sub/rural_All THEN probability of class 1: 99.2% (97.1%-100.0%)
ELSE IF weighting_95_0.9629000000000001_to_inf THEN probability of class 1: 50.0% (2.5%-97.5%)
ELSE probability of class 1: 50.0% (2.5%-97.5%)

rules None


 67%|██████▋   | 2/3 [14:47<07:21, 441.15s/it]

acc 0.5516483516483517
Trained RuleListClassifier 
IF total_acts_at_school_-inf_to_1.5 AND my_check_of_wv_0.0 THEN probability of class 1: 2.0% (1.2%-3.2%)
ELSE IF any_victimization_0.0 AND total_school_beatings_1.0 THEN probability of class 1: 3.6% (2.2%-5.4%)
ELSE IF you_gang_fight_1.5_to_inf AND non-exp_py_alcohol_0.0 THEN probability of class 1: 10.2% (8.5%-12.1%)
ELSE IF you_gang_fight_1.5_to_inf AND anyone_childhood_count_on_All THEN probability of class 1: 25.8% (20.8%-31.2%)
ELSE IF other_race_w/o_refused_0.0 THEN probability of class 1: 99.3% (97.5%-100.0%)
ELSE probability of class 1: 92.3% (73.5%-99.8%)

rules None


100%|██████████| 3/3 [21:49<00:00, 436.46s/it]

acc 0.5901098901098901
Trained RuleListClassifier 
IF you_attack_intent_kill/maim_1.5_to_inf AND my_check_of_wv_0.0 THEN probability of class 1: 1.5% (0.8%-2.5%)
ELSE IF any_victimization_0.0 AND seen_beaten_punched_-inf_to_1.5 THEN probability of class 1: 4.1% (2.8%-5.7%)
ELSE IF non-exper_past_year_illicit_0.0 AND you_gang_fight_1.5_to_inf THEN probability of class 1: 13.3% (11.3%-15.3%)
ELSE IF race_All AND you_broken_+_entering_1.5_to_inf THEN probability of class 1: 91.0% (85.8%-95.1%)
ELSE IF beaten_with_fist/hurt_badly_1.5_to_inf AND friends_use_mar/hash_1.5_to_inf THEN probability of class 1: 75.0% (29.2%-99.2%)
ELSE probability of class 1: 98.3% (93.8%-100.0%)

rules None


In [ ]:
np.random.seed(13)
for i in tqdm(range(3)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43 + i)
    m = imodels.BayesianRuleListClassifier(max_iter=5000, minsupport=0.12, random_state=43 + i)
    m.fit(X_train, y_train, feature_names=X_train.columns)
    print('acc', accuracy_score(y_test, m.predict(X_test)))
    print('rules', print(m))

 33%|███▎      | 1/3 [07:48<15:36, 468.45s/it]

acc 0.6120879120879121
Trained RuleListClassifier 
IF seen_beaten_punched_1.5_to_inf AND fr_suggest_agnts_law_1.5_to_inf THEN probability of class 1: 1.2% (0.5%-2.0%)
ELSE IF any_victimization_0.0 AND non-exp_past_year_marijuana_0.0 THEN probability of class 1: 3.4% (2.2%-4.8%)
ELSE IF you_gang_fight_1.5_to_inf AND you_strong-arm_robbery_1.5_to_inf THEN probability of class 1: 13.5% (11.5%-15.6%)
ELSE probability of class 1: 99.5% (98.2%-100.0%)

rules None


 67%|██████▋   | 2/3 [15:05<07:30, 450.23s/it]

acc 0.9351648351648352
Trained RuleListClassifier 
IF my_check_of_wv_0.0 AND friends_stolen_less_$5_1.5_to_inf THEN probability of class 1: 0.7% (0.2%-1.5%)
ELSE IF you_gang_fight_1.5_to_inf AND you_stolen_motor_veh_1.5_to_inf THEN probability of class 1: 8.8% (7.6%-10.1%)
ELSE IF total_school_muggings_0.0 THEN probability of class 1: 99.4% (98.0%-100.0%)
ELSE probability of class 1: 90.0% (66.4%-99.7%)

rules None
